In [6]:
# 1. Визначити номер варіанта за формулою:

print("Variant number:", ord("M") % 5 + 1)

Variant number: 3


In [7]:
import pandas as pd
import cv2
from PIL import Image, ImageDraw
from IPython.display import display, HTML
import glob
import os

In [8]:
df = pd.read_excel("lab5.xlsx")

class ImageData:          
    def __init__(self, data: pd.Series):
        self.filename = data["file name"]
        self.image_sizes =[int(data) for data in data["image size"].split("x")]
        self.glasses_color = data["glasses color"]
        self.line_width = int(data["line width"])
        
    def __str__(self):
        return f"filename: {self.filename}, image size: {self.image_sizes}, glasses color: {self.glasses_color}, line width: {self.line_width}"

image = ImageData(df[df["N"] == 3].iloc[0])

image.__str__()

'filename: draco.jpg, image size: [700, 700], glasses color: cyan, line width: 4'

In [ ]:
from typing import Tuple, List
import cv2
from PIL import Image, ImageDraw


class Eye:
    def __init__(self, x: int, y: int, w: int, h: int):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.center = (x + w // 2, y + h // 2)
        self.radius = min(w, h) // 2


def detect_face(image: Image) -> Tuple[int, int, int, int]:
    """Detect the largest face in the image."""
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    )
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100)
    )
    
    if faces.size == 0:  # Explicitly check if no faces are detected
        raise ValueError("No faces found in the image.")
    
    return max(faces, key=lambda face: face[2] * face[3])  # Return the largest face


def detect_eyes(image: Image) -> List[Eye]:
    """Detect eyes in the image and return a list of Eye objects."""
    eye_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_eye.xml"
    )
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detected_eyes = eye_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30)
    )
    sorted_eyes = sorted(detected_eyes, key=lambda eye: eye[0])  # Sort by x-coordinate
    return [Eye(x, y, w, h) for (x, y, w, h) in sorted_eyes]


def crop_image(image: Image, center: Tuple[int, int], sizes: Tuple[int, int]) -> Image:
    """Crop the image to the specified size centered around the given point."""
    x, y = center
    w, h = sizes
    half_width, half_height = w // 2, h // 2
    x1, y1 = max(0, x - half_width), max(0, y - half_height)
    x2, y2 = min(image.shape[1], x + half_width), min(image.shape[0], y + half_height)
    return image[y1:y2, x1:x2]


def draw_glasses(
    image: Image, eyes: List[Eye], color: str, line_width: int
) -> Image:
    """Draw glasses around detected eyes."""
    result_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(result_image)

    for eye in eyes:
        draw.ellipse(
            [
                (eye.center[0] - eye.radius, eye.center[1] - eye.radius),
                (eye.center[0] + eye.radius, eye.center[1] + eye.radius),
            ],
            outline=color,
            width=line_width,
        )

    if len(eyes) == 2:  # Connect the two eyes with a line
        draw.line(
            [
                (eyes[0].center[0] + eyes[0].radius, eyes[0].center[1]),
                (eyes[1].center[0] - eyes[1].radius, eyes[1].center[1]),
            ],
            fill=color,
            width=line_width,
        )

    return result_image


def process_image(
    path: str, data: "ImageData", output_path: str = None
) -> str:
    """Process an image by detecting a face, cropping, and drawing glasses."""
    if output_path is None:
        output_path = f"{path}/result_{data.filename}"

    image = cv2.imread(f"{path}/{data.filename}")
    x, y, w, h = detect_face(image)
    cropped_image = crop_image(image, (x + w // 2, y + h // 2), data.image_sizes)
    eyes = detect_eyes(cropped_image)
    result_image = draw_glasses(cropped_image, eyes, data.glasses_color, data.line_width)
    result_image.save(output_path)

    return output_path


# Process images from a DataFrame
for _, row in df.iterrows():
    image_data = ImageData(row)
    process_image("./images", image_data)


In [10]:
original_images = glob.glob("./images/*.*")
original_images = [img for img in original_images if not os.path.basename(img).startswith("result_")]

# Create HTML table
html = """
<table style="width: 100%; border-collapse: collapse;">
    <tr>
        <th style="text-align: center; padding: 10px;">Before</th>
        <th style="text-align: center; padding: 10px;">After</th>
    </tr>
"""

for orig_path in original_images:
    filename = os.path.basename(orig_path)
    result_path = f"./images/result_{filename}"
    
    if os.path.exists(result_path):
        # Open images to get dimensions for proper sizing
        orig_img = Image.open(orig_path)
        result_img = Image.open(result_path)
        
        # Calculate display size (you can adjust the max_width)
        max_width = 400
        ratio = max_width / max(orig_img.width, result_img.width)
        display_width = int(orig_img.width * ratio)
        
        html += f"""
        <tr>
            <td style="text-align: center; padding: 10px;">
                <img src="{orig_path}" width="{display_width}" style="max-width: 100%;">
            </td>
            <td style="text-align: center; padding: 10px;">
                <img src="{result_path}" width="{display_width}" style="max-width: 100%;">
            </td>
        </tr>
        """

html += "</table>"
display(HTML(html))

Before,After
,
,
,
,
,
